In [1]:
from collections import defaultdict
import json, os, _pickle as pickle, sys
#  读取实体字典，获得其中的 med_synonyms_name_dict字典，用于实体归一
def loads_pick(map):

#    if os.path.exists(map):
    fs = open(map, 'rb')
    med_synonyms_name_dict, med_name_cate_dict = pickle.loads(fs.read())  # 使用loads反序列化
#    med_name_synonyms_dict = {v:k for k,v in med_synonyms_name_dict.items()}
#    else:
#        print ('not exist dict.pkl')
#        exit(0)
    return med_synonyms_name_dict

#  将两个库中的数据合在一个文件里  （例：将 malacards 和 drugbank 的数据和在一个文件里，单独运行）
def file_add(file1,file2):
    with open(file1,mode='r') as file,open(file2, 'a') as fl:
        
        file.seek(0,0)
        line = file.readline()
        
        while(line):
            line_dict = json.loads(line.strip())        
            fl.write(json.dumps(line_dict) + '\n')
            
            line = file.readline()
        fl.close()

#  将关系字典中的实体归一，并将关系库的名称均用数字来代替    relation_dict_before_guiyi_v2.0   relation_dict_after_guiyi_v2.0
def e1_e2_guiyi(file_name, med_synonyms_name_dict):
    with open('./all_relation_dict_file_v2.0/relation_dict_before_guiyi_v2.0/'+file_name) as fr , open('./all_relation_dict_file_v2.0/relation_dict_after_guiyi_v2.0/'+file_name,'a') as fw:
        check_d=[]
        num_dict = {"ttd": "-1", "kegg": "-2", "dgidb": "-3", "drugbank": "-4", "brenda": "-5", \
                    "uniprot": "-6", "chembl": "-7", "drugcentral": "-8", "malacards": "-9", "ctd": "-10", \
                    "biogrid": "-11", "ebi": "-12", "omim": "-13", "disgenet": "-14", "gene_organizer": "-15", \
                    "-1":"-1", "-2":"-2", "-3":"-3", "-4":"-4", "-5":"-5", "-6":"-6", "-7":"-7", "-8":"-8", \
                    "-9":"-9", "-10":"-10", "-11":"-11","-12":"-12", "-13":"-13", "-14":"-14", "-15":"-15"}
#        for n,i in enumerate(fr.readlines()):
        for line in fr:
            line_dict = json.loads(line.strip())
            flag1 = False; flag2 = False            
#            if (n==0):
#                print (sys.getsizeof(fr))
#            try:
#            json_text = json.loads(i.strip())
            e1 = line_dict['e1'].lower()
            e2 = line_dict['e2'].lower()
            source = line_dict["sources"]
            line_dict["sources"] = list(set([num_dict[v] for v in source]))
            if (e1 in med_synonyms_name_dict):   #关系字典中的实体是实体字典synonyms_name_dict中的key,则将实体修改为name
                line_dict["e1"] = med_synonyms_name_dict[e1]
                flag1 = True                    

            if (e2 in med_synonyms_name_dict):   #关系字典中的实体是实体字典synonyms_name_dict中的key,则将实体修改为name
                line_dict["e2"] = med_synonyms_name_dict[e2]
                flag2 = True

            if flag1 and flag2 :
                check_d.append([e1,e2])
            fw.write(json.dumps(line_dict) + '\n')
#                yield json_text
                ##去重
                #fw.write(json.dumps(json_text))
                #fw.write('\n')
#            except ValueError:
                #logging.info('json format error')
#                print ('json format errot®')
        print('len(check_id):',len(check_d))

#  将归一之后关系字典中实体既属于 e1 又属于 e2 的关系删除，同时删除实体长度小于等于2的关系对    relation_dict_after_guiyi   relation_dict_self_align1
def entity_belong_e1_and_e2(file_name):   #  注意：像 gene_gene 这样的文件就不需要处理了
    print("delete_entity_belong_e1_and_e2:")
    with open('./all_relation_dict_file_v2.0/relation_dict_after_guiyi_v2.0/' + file_name,'r') as file, open('./all_relation_dict_file_v2.0/relation_dict_self_align1_v2.0/' + file_name, 'w') as fl:
        e1_set = set()
        e2_set = set()
        del_data = 0
        del_data_list = []
        file.seek(0,0)
        for line in file:
            line_dict = json.loads(line.strip())
            e1 = line_dict["e1"].lower()
            e2 = line_dict["e2"].lower()
            e1_set.add(e1)
            e2_set.add(e2)

        file.seek(0,0)
        for line in file:
            line_dict = json.loads(line.strip())
            e1 = line_dict["e1"].lower()
            e2 = line_dict["e2"].lower()
            if (e1 in e2_set) or (e2 in e1_set) or (len(e1) <= 2) or (len(e2) <= 2):
                del_data += 1
#                del_data_list.append(e1 + "##" + e2)
#                print(e1 + "##" + e2)
                continue
            else:
#                print(line_dict)
                fl.write(json.dumps(line_dict) + '\n')
        print(file_name + ":" + str(del_data))
        
#  合并 e1、e2 相同的实体              relation_dict_self_align1    combine_relation_dict
def combine_relation(file_name):

#for rfile in rfile_list:
#    with open('./all_relation_dict_file_v2.0/relation_dict_self_align1_v2.0/' + file_name,mode='r') as file,open('./all_relation_dict_file_v2.0/combine_relation_dict_v2.0/' + file_name,'w+') as fl:   # 关系字典与实体字典对齐之前用这个路径
    with open('./relation_entity_dict_align_v3.1/' + file_name,mode='r') as file,open('./all_relation_dict_file_v2.0/combine_relation_dict_v2.1/' + file_name,'w+') as fl:  # 关系字典与实体字典对齐之后用这个路径
        file.seek(0,0)
        line = file.readline()
        res = {}
        while(line):
            line_dict = json.loads(line.strip())
            e1 = line_dict["e1"].lower()
            e2 = line_dict["e2"].lower()
            score = line_dict["scores"]
    #            if score ==[]:
    #                score = [""]

            pid = line_dict["pids"]
    #            if pid ==[]:
    #                pid = [""]                
            source = line_dict["sources"]

            name_combine = e1 + '####' + e2
            if name_combine not in res:
                res[name_combine] = {"e1":e1,"e2":e2,"scores":score,"pids":pid,"sources":source}
            else:
                res[name_combine]["scores"].extend(score)
                res[name_combine]["pids"].extend(pid)
                res[name_combine]["sources"].extend(source)

            line = file.readline()

        for value in res.values():
            fl.write(json.dumps(value) + '\n')
        fl.close()

#  将关系中的每个字段都做去重处理                combine_relation_dict    quchong_relation_dict
def quchong_each_field(file_name):            #  将所有文件中的数据去重
    with open('./all_relation_dict_file_v2.0/combine_relation_dict_v2.1/'+file_name, 'r') as file, open('./all_relation_dict_file_v2.0/quchong_relation_dict_v2.1/'+file_name, 'w') as fl:
        file.seek(0,0)
        res = {}
#        dict_qu = {}
        list_qu = []
        for i,line in enumerate(file):
            if i % 100000 ==0:print(i)
            line_dict = json.loads(line.strip())
#            res = {"e1":line_dict["e1"],"e2":line_dict["e2"],"scores":list(set(line_dict["scores"])),"pids":list(set(line_dict["pids"])),"sources":list(set([num_dict[v] for v in line_dict["sources"]]))}
            res = {"e1":line_dict["e1"],"e2":line_dict["e2"],"scores":list(set(line_dict["scores"])),"pids":list(set(line_dict["pids"])),"sources":list(set(line_dict["sources"]))}

#            dict_qu[json.dumps(res)] = 1
            list_qu.append(json.dumps(res))
        list_quchong = list(set(list_qu))
        for key in list_quchong:
            fl.write(key + "\n")             
        fl.close()
#  todo:解决关系字典的自对齐问题（包含：单个关系字典的自对齐，关系字典之间的对齐）  relation_dict_self_align2
#  todo:解决关系字典与实体字典的歧义问题(代码在本机的文件中)
#  todo:解决完歧义问题，仅需要跑 “合并 e1,e2”;“去重” 这两个部分代码～

if __name__=='__main__':
#    file_add('./genes_diseases_cause_ctd.dict', \
#             './gene_disease_cause_old.dict')

    map = './medical_entity_dict_v4.4.pkl'
    med_synonyms_name_dict = loads_pick(map)
#    rfile_list = ['compound_protein_treat.dict','disease_compound_cause.dict','disease_disease_coexist.dict', \
#                  'drugs_disease_treat.dict','drugs_gene_target.dict','drugs_protein_target.dict','gene_anatomy_locate.dict', \
#                  'gene_compound_treat.dict','gene_gene_interact.dict','gene_protein_express.dict', \
#                  'protein_disease_cause.dict','protein_organism_belong.dict','protein_protein_interact.dict']
#    rfile_list = ['gene_disease_cause.dict']    #  单独跑：'gene_disease_cause.dict',

#    file_folder_path = r"./all_relation_dict_file_v2.0/relation_dict_before_guiyi_v2.0"  #  第一遍跑用该路径
    file_folder_path = r"./relation_entity_dict_align_v3.1"       #  第二编跑用该路径
    
    file_path_list = []
    for file_path_root,_,file_name_list in os.walk(file_folder_path):
        for file_name in file_name_list:
#            if ("compound_disease_cause.dict" in file_name) or ("compound_gene_treat.dict" in file_name) \
#                or ("compound_protein_treat.dict" in file_name):
            if ".dict" in file_name:
                file_path_list.append(file_name)
    num = 0
    for file_name in file_path_list:
        num = num + 1
        print(str(num) + ":" + file_name)
        e1_e2_guiyi(file_name,med_synonyms_name_dict)  #  将关系字典中的实体归一，并将关系库的名称均用数字来代替 
        entity_belong_e1_and_e2(file_name)           #  将归一之后关系字典中实体既属于 e1 又属于 e2 的关系删除
        combine_relation(file_name)                #  合并 e1、e2 相同的实体
        quchong_each_field(file_name)            #  将关系中的每个字段都做去重处理
    
    

1:compound_protein_treat.dict
0
100000
2:gene_organism_belong.dict
0
3:equipment_disease_diagnose|treat.dict
0
4:drugs_anatomy_affect.dict
0
5:compound_disease_cause.dict
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
6:gene_disease_cause.dict
0
100000
7:gene_protein_express.dict
0
8:drugs_organism_apply.dict
9:disease_anatomy_occur.dict
0
10:compound_drugs_contain.dict
0
11:protein_anatomy_belong.dict
0
12:gene_gene_interact.dict
0
100000
200000
300000
400000
500000
600000
700000
800000
13:gene_anatomy_locate.dict
0
100000
14:drugs_disease_treat.dict
0
100000
200000
300000
400000
500000
15:disease_disease_coexist.dict
0
100000
200000
300000
400000
16:drugs_protein_target.dict
0
100000
17:protein_protein_interact.dict
0
18:protein_disease_cause.dict
0
19:protein_organism_belong.dict
0
20:drugs_gene_target.dict
0
100000
200000
300000
21:drugs_drugs_interact.dict
0
22:compound_gene_treat.dict
0
100000
200000
300000
400000
500000
600000
70